# Procesamiento de Lenguaje Natural
## Desafío 2

- Tomar un ejemplo de los bots utilizados (uno de los dos) y construir el propio.
- Sacar conclusiones de los resultados.

IMPORTANTE: Recuerde para la entrega del ejercicio debe quedar registrado en el colab las preguntas y las respuestas del BOT para que podamos evaluar el desempeño final.

In [5]:
import json
import string
import random
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('punkt_tab') # <-- CORRECCIÓN: Añadido para solucionar el LookupError
nltk.download('wordnet')
nltk.download('omw-1.4')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Creo un archivo intents.json con las intenciones, patrones y respuestas para nuestro bot. La idea es armar un bot de información básica para la facultad.

In [2]:
# defino el contenido del JSON como un string
json_content = """
{
  "intents": [
    {
      "tag": "saludo",
      "patterns": ["Hola", "Como estas?", "Hay alguien ahi?", "Buen dia", "Buenas tardes"],
      "responses": ["Hola!", "Hola, ¿en qué puedo ayudarte?", "¡Buen día! ¿Qué necesitas saber?"]
    },
    {
      "tag": "despedida",
      "patterns": ["Chau", "Adios", "Hasta luego", "Nos vemos", "Tengo que irme"],
      "responses": ["¡Hasta luego! Que tengas un buen día.", "Adiós, ¡vuelve pronto!", "Chau!"]
    },
    {
      "tag": "agradecimiento",
      "patterns": ["Gracias", "Muchas gracias", "Te lo agradezco", "Genial, gracias"],
      "responses": ["¡De nada!", "Es un placer ayudarte.", "¡No hay problema!"]
    },
    {
      "tag": "nombre",
      "patterns": ["Quien sos?", "Como te llamas?", "Cual es tu nombre?"],
      "responses": ["Soy un bot de asistencia de la facultad, creado para resolver dudas.", "Mi nombre es FIUBABot, ¿en qué te ayudo?"]
    },
    {
      "tag": "inscripciones",
      "patterns": ["Cuando son las inscripciones?", "Como me inscribo a materias?", "Fechas de inscripcion", "inscripcion a finales"],
      "responses": ["Las fechas de inscripción a materias suelen ser las semanas previas al inicio del cuatrimestre. Puedes consultar el calendario académico en el sitio web de la facultad.", "Para inscripciones a finales, revisa el sistema SIU Guaraní."]
    },
    {
      "tag": "ubicacion",
      "patterns": ["Donde queda la facultad?", "Como llego?", "Direccion de la FIUBA", "Sede Las Heras", "Sede Paseo Colon"],
      "responses": ["La facultad tiene dos sedes principales: Av. Paseo Colón 850 y Av. Las Heras 2214.", "La sede de Paseo Colón es la histórica. La sede de Las Heras es más nueva."]
    },
    {
      "tag": "carreras",
      "patterns": ["Que carreras hay?", "Oferta academica", "Quiero estudiar ingenieria", "carreras de grado"],
      "responses": ["La facultad ofrece 12 carreras de grado, incluyendo Ingeniería Informática, Civil, Industrial, Mecánica, Electrónica, y más. También ofrece carreras de posgrado."]
    }
  ]
}
"""

# Escribimos el contenido al archivo
with open('intents.json', 'w', encoding='utf-8') as f:
    f.write(json_content)

print("Archivo 'intents.json' creado exitosamente.")

Archivo 'intents.json' creado exitosamente.


Cargamos el JSON y procesamos los patrones. Vamos a utilizar tokenización y lematización con NLTK

In [6]:
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

# Cargar el archivo JSON
with open('intents.json', 'r', encoding='utf-8') as f:
    intents = json.load(f)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenizar
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        # Añadir a documentos (patrón, tag)
        documents.append((word_list, intent['tag']))
        # Añadir el tag a la lista de clases (si no está ya)
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lematizar, pasar a minúsculas y quitar puntuación
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words))) # Eliminar duplicados y ordenar

classes = sorted(list(set(classes)))

# Guardar words y classes en archivos pickle para usarlos luego
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

print("Palabras (vocabulario):", len(words))
print("Clases (intents):", len(classes))
print("\nDocumentos (patrones):", len(documents))

Palabras (vocabulario): 61
Clases (intents): 7

Documentos (patrones): 30


Convvierto nuestros patrones de texto en vectores numéricos usando el modelo Bag-of-Words (BoW) y creamos las etiquetas de salida (output) como vectores *one-hot*.

In [7]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    # Inicializar Bag of Words
    bag = []
    # Lista de patrones tokenizados
    pattern_words = doc[0]
    # Lematizar cada palabra
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # Crear el vector BoW
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output_row es el vector one-hot
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Mezclar los datos
random.shuffle(training)

# Separar en X (features) e Y (labels)
train_x = np.array([i[0] for i in training])
train_y = np.array([i[1] for i in training])

print("Datos de entrenamiento (X) creados. Shape:", train_x.shape)
print("Datos de salida (Y) creados. Shape:", train_y.shape)

Datos de entrenamiento (X) creados. Shape: (30, 61)
Datos de salida (Y) creados. Shape: (30, 7)


Definimos una red neuronal secuencial simple con Keras para clasificar las intenciones.

In [8]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compilar el modelo
# Usamos SGD (Descenso de Gradiente Estocástico)
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Entrenar el modelo
hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Guardar el modelo
model.save('chatbot_model.h5', hist)

print("\nModelo entrenado y guardado como 'chatbot_model.h5'.")

Epoch 1/200


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.2486 - loss: 1.9817
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2367 - loss: 1.9076 
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1276 - loss: 1.9270     
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3343 - loss: 1.8693 
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1238 - loss: 1.9283     
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3686 - loss: 1.7444
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2267 - loss: 1.7739     
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4581 - loss: 1.7215 
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3610 - loss: 1.7432 
Epoch 10/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4024 - loss: 1.6654 
Epoch 11/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3062 - loss: 1.7094 
Epoch 12/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3481 - loss


Modelo entrenado y guardado como 'chatbot_model.h5'.


Creamos las funciones necesarias para procesar la entrada del usuario, predecir la intención usando el modelo entrenado y devolver una respuesta.

In [9]:
# Cargar los archivos pickle
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = tf.keras.models.load_model('chatbot_model.h5')

# Función para preprocesar la entrada del usuario
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Función para convertir la entrada en Bag of Words
def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return(np.array(bag))

# Función para predecir la clase (intent)
def predict_class(sentence, model):
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25 # Umbral de confianza
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    # Ordenar por probabilidad
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    # Si no supera el umbral, devolver una respuesta por defecto
    if not return_list:
        return_list.append({"intent": "sin_respuesta", "probability": "1.0"})

    return return_list

# Función para obtener la respuesta
def get_response(ints, intents_json):
    tag = ints[0]['intent']

    if tag == "sin_respuesta":
        return "Lo siento, no entendí tu pregunta. ¿Puedes reformularla?"

    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

# Función principal del chat
def chatbot_response(text):
    ints = predict_class(text, model)
    res = get_response(ints, intents)
    return res

Aca registramos las preguntas y respuestas del BOT para evaluar su desempeño, tal como solicita la consigna.

In [10]:

print("Iniciando chat con FIUBABot (escribe 'salir' para terminar)...")
print("-" * 50)

# Lista de preguntas para probar
preguntas_prueba = [
    "Hola",
    "como te llamas?",
    "donde queda la facultad?",
    "que carreras tienen?",
    "cuando me puedo inscribir?",
    "muchas gracias",
    "como llego a paseo colon?",
    "que es la ingenieria?", # Pregunta fuera de contexto
    "adios"
]

# Iterar sobre las preguntas de prueba y registrar las respuestas
for pregunta in preguntas_prueba:
    respuesta = chatbot_response(pregunta)
    print(f"Pregunta: {pregunta}")
    print(f"Respuesta: {respuesta}")
    print("-" * 50)

Iniciando chat con FIUBABot (escribe 'salir' para terminar)...
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Pregunta: Hola
Respuesta: ¡Buen día! ¿Qué necesitas saber?
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Pregunta: como te llamas?
Respuesta: Mi nombre es FIUBABot, ¿en qué te ayudo?
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Pregunta: donde queda la facultad?
Respuesta: La facultad tiene dos sedes principales: Av. Paseo Colón 850 y Av. Las Heras 2214.
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Pregunta: que carreras tienen?
Respuesta: La facultad ofrece 12 carreras de grado, incluyendo Ingeniería Informática, Civil, Industrial, Mecánica, Electrónica, y más. También ofrece carreras de posgrado.
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
Pregunta: cuando me p

### Análisis y Conclusiones

El modelo implementado, basado en Bag-of-Words (BoW) y una Red Neuronal simple, demostró ser efectivo para clasificar un conjunto de intenciones simples y bien definidas. Respondió correctamente a interacciones sociales (saludos, despedidas) y consultas informativas (ubicación, carreras, inscripciones) siempre que la entrada del usuario se pareciera a los patrones de entrenamiento. Un aspecto clave de su funcionamiento fue el manejo de la incertidumbre: gracias al ERROR_THRESHOLD, el bot pudo identificar preguntas fuera de su vocabulario (como "¿qué es la ingeniería?") y devolver una respuesta por defecto en lugar de una incorrecta.

Sin embargo, el bot presenta limitaciones claras. Su principal debilidad es la sensibilidad al vocabulario, ya que el modelo BoW no puede entender sinónimos; si un usuario utiliza una palabra no registrada (p.ej., "anotarme" en lugar de "inscribirme"), el bot fallará. Además, carece de memoria contextual, procesando cada pregunta de forma aislada. Estas limitaciones podrían abordarse añadiendo muchos más patrones de entrenamiento para cada intención o, de forma más robusta, reemplazando BoW con Word Embeddings (como los vistos en 2b y 2c con Gensim o GloVe), lo que permitiría al modelo entender relaciones semánticas y sinónimos.